# VannaAI

Es una librería de python que permite crear código SQL a partir de texto usando algún modelo de AI.

Esto se hace mediante un arquitectura llamada RAG, la cual requiere mínimo:

- Base de datos vectorizada: donde se almacena la documentación relevante para el contexto.
- LLM: lo que en este caso crea el código SQL usando la documentación obtenida del elemento anterior.



GitHub: https://github.com/vanna-ai/vanna

Docs: https://vanna.ai/docs/index.html


In [ ]:
import sqlite3
import pandas as pd
from vanna.remote import VannaDefault

In [ ]:
# https://vanna.ai/account/profile
api_key = '' # api key que entrega vanna para el uso de LLM
vanna_model_name = 'test_de' # el nombre de la base de datos
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

Para "entrenar" podemos utilizar 3 cosas:

- DDL: definición de las tablas.
- Question/SQL: Pregunta con su respectiva respuesta en SQL.
- Documentación: contexto adicional a modo de texto.

In [15]:
vn.train(
    ddl="""
CREATE TABLE clientes (
    cliente_id    INTEGER PRIMARY KEY,
    nombre        TEXT    NOT NULL,
    ciudad        TEXT,
    edad          INTEGER,
    fecha_registro DATE
)
"""
)

Adding ddl: 
CREATE TABLE clientes (
    cliente_id    INTEGER PRIMARY KEY,
    nombre        TEXT    NOT NULL,
    ciudad        TEXT,
    edad          INTEGER,
    fecha_registro DATE
)



Exception: ('Error adding DDL', {'error': {'code': -32600, 'message': 'Invalid Request: No key'}, 'jsonrpc': '2.0'})

In [5]:
vn.train(
    ddl="""
CREATE TABLE pedidos (
    pedido_id      INTEGER PRIMARY KEY,
    cliente_id     INTEGER,
    producto       TEXT    NOT NULL,
    cantidad       INTEGER NOT NULL,
    precio_unit    REAL    NOT NULL,
    fecha_pedido   DATE    NOT NULL,
    estado         TEXT    NOT NULL,
    FOREIGN KEY(cliente_id) REFERENCES clientes(cliente_id)
)
"""
)

Adding ddl: 
CREATE TABLE pedidos (
    pedido_id      INTEGER PRIMARY KEY,
    cliente_id     INTEGER,
    producto       TEXT    NOT NULL,
    cantidad       INTEGER NOT NULL,
    precio_unit    REAL    NOT NULL,
    fecha_pedido   DATE    NOT NULL,
    estado         TEXT    NOT NULL,
    FOREIGN KEY(cliente_id) REFERENCES clientes(cliente_id)
)



'15462843-ddl'

In [6]:
vn.train(
    documentation="El usuario puede usar el nombre de orden en lugar de pedido."
)

Adding documentation....


'3436304-doc'

In [7]:
vn.train(
    documentation="Los posibles estados de un pedido son: ENVIADO, ENTREGADO, PENDIENTE, CANCELADO."
)

Adding documentation....


'3436305-doc'

In [8]:
vn.train(
    question="Cómo se clasifica el nivel de gasto de cada cliente en “ALTO”, “MEDIO” o “BAJO”, según el total que ha gastado?",
    sql="""
WITH gastos AS (
  SELECT cliente_id,
         SUM(cantidad * precio_unit) AS monto_total
    FROM pedidos
   GROUP BY cliente_id
)
SELECT c.nombre,
       g.monto_total,
       CASE
         WHEN g.monto_total > 1000 THEN 'ALTO'
         WHEN g.monto_total BETWEEN 300 AND 1000 THEN 'MEDIO'
         ELSE 'BAJO'
       END AS categoria
  FROM gastos AS g
  JOIN clientes AS c ON c.cliente_id = g.cliente_id;
"""
)

'254ebadffd85e71abcb97cae617348bb-sql'

Para visualizar lo almacenado hasta el momento:

In [9]:
vn.get_training_data()

,id,training_data_type,question,content
0,1220660-sql,sql,Cómo se clasifica el nivel de gasto de cada cl...,"\nWITH gastos AS (\n SELECT cliente_id,\n ..."
1,3436305-doc,documentation,None,Los posibles estados de un pedido son: ENVIADO...
2,15462843-ddl,ddl,None,\nCREATE TABLE pedidos (\n pedido_id I...
3,3436304-doc,documentation,None,El usuario puede usar el nombre de orden en lu...
4,15462827-ddl,ddl,None,\nCREATE TABLE clientes (\n cliente_id I...


Para eliminar algo de la base de datos:

In [8]:
vn.remove_training_data('15462827-ddl') # se ingresa el id que se visualiza en la tabla anterior

True

Para generar SQL:

In [15]:
query = vn.generate_sql(
    "cual fue la ultima orden pedida?"
)

SQL Prompt: [{'role': 'system', 'content': "You are a SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\nCREATE TABLE pedidos (\n    pedido_id      INTEGER PRIMARY KEY,\n    cliente_id     INTEGER,\n    producto       TEXT    NOT NULL,\n    cantidad       INTEGER NOT NULL,\n    precio_unit    REAL    NOT NULL,\n    fecha_pedido   DATE    NOT NULL,\n    estado         TEXT    NOT NULL,\n    FOREIGN KEY(cliente_id) REFERENCES clientes(cliente_id)\n)\n\n\n\nCREATE TABLE clientes (\n    cliente_id    INTEGER PRIMARY KEY,\n    nombre        TEXT    NOT NULL,\n    ciudad        TEXT,\n    edad          INTEGER,\n    fecha_registro DATE\n)\n\n\n\n===Additional Context \n\nEl usuario puede usar el nombre de orden en lugar de pedido.\n\nLos posibles estados de un pedido son: ENVIADO, ENTREGADO, PENDIENTE, CANCELADO.\n\n===Response Guidelines \n

In [13]:
print(query)

SELECT *
  FROM pedidos
 ORDER BY fecha_pedido DESC
 LIMIT 1;


In [ ]:
conn = sqlite3.connect('db_basic_queries.sqlite3')

In [ ]:
pd.read_sql_query(query, conn)

,pedido_id,cliente_id,producto,cantidad,precio_unit,fecha_pedido,estado
0,105,4,Smartphone,1,1200.0,2025-07-02,ENVIADO
